In [1]:
from akorn import AKORN
from util import *
import matplotlib.pyplot as plt
from comps.local_poly_regression import *
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [54]:
T = 1000
X = np.linspace(0, T, T)/T

std = 0.3
var = std**2

G = doppler(abs(1 - X), 0.38)
Y = np.random.normal(G, std, T)

ak = AKORN(X, Y, var)
ak.train()

lr =  0.040171561812911066
run


In [55]:
from flh import *
online = FLH(X, Y, T)
online.run()

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.00000000e-01, 5.00000000e-01, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [1.12235872e-80, 1.08193790e-80, 9.51652896e-81, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [9.47155048e-81, 9.13286085e-81, 8.04028982e-81, ...,
        1.00200401e-03, 0.00000000e+00, 0.00000000e+00],
       [9.91295045e-81, 9.55879891e-81, 8.41621700e-81, ...,
        8.45520251e-04, 1.00100100e-03, 0.00000000e+00]])

In [18]:
## Get (approximate) best bw for ll
# min_bw = 0.01
# bandwidths = [min_bw*(2**i) for i in range(2+int(math.log(1/min_bw, 2)))]

# best_bw = 0
# best_bw_mse = float("inf")

# Y_local = None
# for bw in bandwidths:
#     Y_local_bw = np.array([local_linear(X, Y, bw, x, kernel_func = boxcar) for x in X])
#     mse = ((G - Y_local_bw)**2).mean()
#     print("bw = ", bw, "    Mse = ", mse)
#     if mse < best_bw_mse:
#         Y_local = Y_local_bw
#         best_bw_mse = mse
#         best_bw = bw
        
best_bw = 0.02 #the output of the above code, which takes a long time to run.


In [56]:
H_akorn = ak.get_hat_matrix()
H_online = online.get_hat_matrix()
H_ll = local_linear_hatmatrix_2(X, best_bw, gaussian)

In [57]:
from util import *

level = 40

H_akorn = log_transform(H_akorn, level)
H_online = log_transform(H_online, level)
H_ll = log_transform(H_ll, level)

In [68]:
t = T
# fig, ax = subplots_centered(nrows=2, ncols=2, figsize=(10,7), nfigs=3)
fig, ax = plt.subplots(1, 4)
# fig.suptitle("Large Values of Hat Matrices", fontsize = 20)

ax00 = ax[0]
ax0 = ax[1]
ax1 = ax[2]
ax2 = ax[3]

ax00.set_xticks(ticks=[], labels=[])
ax00.set_yticks(ticks=[], labels=[])
ax0.set_xticks(ticks=[], labels=[])
ax0.set_yticks(ticks=[], labels=[])
ax1.set_xticks(ticks=[], labels=[])
ax1.set_yticks(ticks=[], labels=[])
ax2.set_xticks(ticks=[], labels=[])
ax2.set_yticks(ticks=[], labels=[])

ax1.title.set_text("ADDLE")
ax0.title.set_text("Local Linear Regression")
ax2.title.set_text("AKORN")


cmap = plt.get_cmap('PuOr')

# extract all colors from the .jet map
cmaplist = [cmap(i) for i in reversed(range(cmap.N))]
# create the new map
cmap = cmap.from_list('Custom cmap', cmaplist, cmap.N)


heatmap = ax0.imshow(H_ll, vmin = -np.max(H_akorn)/2, vmax = np.max(H_akorn)/2, cmap=cmap)
ax0.plot(range(0,t), -G[:t]*T/10 +T/2, color = 'black', linewidth = 3)
# plt.show()


heatmap2 = ax1.imshow(H_online, vmin = -np.max(H_akorn)/2, vmax = np.max(H_akorn)/2, cmap = cmap)
ax1.plot(range(0,t), -G[:t]*T/10 +T/2, color = 'black', linewidth = 3)

heatmap2 = ax2.imshow(H_akorn, vmin = -np.max(H_akorn)/2, vmax = np.max(H_akorn)/2, cmap = cmap)
ax2.plot(range(0,T), -G*T/10 +T/2, color = 'black', linewidth = 3)

ax00.plot(X, G, color = "black")
ax00.plot(X, Y, alpha = 0.5, color = "gray")
ax00.set_box_aspect(1)


fig.subplots_adjust(wspace=0.4)


plt.show()